In [ ]:
from transformers import pipeline
model = "siebert/sentiment-roberta-large-english"

analysis = pipeline(task="sentiment-analysis",model=model) 
print(analysis("Something ain't right"))

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

analysis = pipeline(task="sentiment-analysis",model=model,tokenizer=tokenizer) 
a = analysis("When twitter spaces ?")

print(a[0]['label'] + ' ' + str(a[0]['score']))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


neutral 0.9254334568977356


In [85]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np

#https://stackoverflow.com/questions/66160583/removing-nearly-duplicate-observations-python
def dedupe_partially_vectorized(df, threshold=1):
    """
    - Iterate through each row starting from the last; examine all previous rows for duplicates.  
    - If found, it is appended to a list of duplicate indices.
    """
    # convert field data to integers
    enc = OrdinalEncoder()
    X = enc.fit_transform(df[["tweet","user_id"]].to_numpy())



    """
    - loop through each row, starting from last
    - for each `row`, calculate hamming distance to all previous rows
    - if any such distance is `threshold` or less, mark `idx` as duplicate
    - loop ends at 2nd row (1st is by definition not a duplicate)
    """
    dupe_idx = []          
    count = 0
    for j in range(len(X) - 1):

        idx = len(X) - j - 1
        row = X[idx]
        prev_rows = X[0:idx]
        dists = np.sum(row != prev_rows, axis=1)
        if min(dists) <= threshold:
            dupe_idx.append(idx)
            count+=1
        dupe_idx = sorted(dupe_idx)
    df_dupes = df.iloc[dupe_idx]
    df_deduped = df.drop(df_dupes.index,axis=0)

    print("Number of counts" + str(count))
    return df_deduped,df_dupes

In [2]:
import difflib

dupe_df = None
def dedupe(df,threshold=0.85):
    dupe = df.tweet.apply(lambda row: difflib.get_close_matches(row, list(df[df.tweet!=row].tweet), cutoff = threshold))    
    df_dupes = dupe[dupe.astype(bool)]
    df_deduped = df.drop(df_dupes.index,axis=0)
    return df_deduped,df_dupes

NameError: name 'df_dupes' is not defined

In [164]:
dupe_df

43     [1. how horny do you have to be to rt porn on ...
51     [1. how horny do you have to be to rt porn on ...
62     [NIKE Hello twitter welcome to rootjazz🌼 806  ...
76     [💝 💝how horny do you have to be to rt porn on ...
84     [1. how horny do you have to be to rt porn on ...
87     [💝 💝how horny do you have to be to rt porn on ...
100    [NIKE Hello twitter welcome to rootjazz🌼 886  ...
105    [1. how horny do you have to be to rt porn on ...
114    [Do you remember when you joined Twitter? I do...
142    [1. how horny do you have to be to rt porn on ...
158    [1. how horny do you have to be to rt porn on ...
163    [1. how horny do you have to be to rt porn on ...
175    [Do you remember when you joined Twitter? I do...
176    [1. how horny do you have to be to rt porn on ...
198    [1. how horny do you have to be to rt porn on ...
215    [💝 💝how horny do you have to be to rt porn on ...
234    [1. how horny do you have to be to rt porn on ...
279    [1. how horny do you hav

1599681264532873216 2022-12-05 16:25:08 +0800 <mafe_jorge> #República_Española en [ f ] captura  5D Republicat a 🐦 Twitter per  📷 J®️GediThor2022 ❤️❤️❤️❤️❤️❤️ 💛💛💛💛💛💛 💜💜💜💜💜💜 #SyR por #CoherenciaDemocrática #FelizSemanaRepublicana  https://t.co/vE5t6EGKdZ
1599681263241396225 2022-12-05 16:25:08 +0800 <br_rvw> Twitter始めました🌸 出稼ぎ紹介してくれるスカウトさん 紹介お願い致します🙇‍♀️
1599681262918463488 2022-12-05 16:25:08 +0800 <konbupis> なんでTwitterに投稿したら音ズレするの🫠
1599681262335119360 2022-12-05 16:25:08 +0800 <C_A_Guardiola> Tonight on Twitter: parts of speech front and center; the lies we tell ourselves about cats; and both Frank and Mark read your Twitter confessional and affirm that they have had enough of your shit.
1599681262003933186 2022-12-05 16:25:08 +0800 <AEON_JAPAN> @chiiichann22  ご応募いただきありがとうございます。 Twitterキャンペーンの結果は…残念😢 キャンペーンは12/25(日)23:59まで！  クリスマスでお買い物を楽しもう🎄  🔽詳細  https://t.co/lj5Xk9mYJh
1599681261919879169 2022-12-05 16:25:08 +0800 <Colambole> @JulienFort4bis @Roxas_Sui @Acermendax @RReichstadt @MrTea4

In [3]:
import twint
import nest_asyncio
nest_asyncio.apply()

# Configure
c = twint.Config()
c.Search = "Twitter"
c.Limit = 200
c.Lang = "en"
c.Pandas = True

# Run
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

#Sub selects english tweets
en_df = Tweets_df.query("language == 'en'")
print("Length of tweets in English: " + str(len(en_df)))


#en_df.drop_duplicates(subset=['tweet'])

#de_df,dupe_df = dedupe_partially_vectorized(en_df)
#Drop rows that are very similar to prevent spam that may affect results
de_df,dupe_df = dedupe(en_df)

#Only select columns that are used
df = de_df[["tweet", "nlikes","nretweets","nreplies"]]



#Calculate traction of tweet
df["traction"]= df["nlikes"] * 2 + df["nreplies"] * 3 + df["nretweets"] * 4

#Sorts the dataframe according to the most number of likes
sorted_df = df.sort_values(by=['traction'], ascending=False)
sorted_df

1601519725502427137 2022-12-10 18:10:32 +0800 <SignHexa> @TomFitton @Twitter Lol. No.
1601519725342695424 2022-12-10 18:10:31 +0800 <AshleyH03872216> how horny do you have to be to rt porn on twitter  #深圳 #深圳线下  https://t.co/AezAY9mf1i
1601519723807928320 2022-12-10 18:10:31 +0800 <a_ura_oO> @Rui__UTXTU 時差リプ失礼っ！ ・にこ ・1月9日 ・強いて言うならTwitter上で ｷﾞｭ(´｡›ω(•ω•｡`)~ｯ♡ とか送ってくる輩 ・いっぱい絡みたいなって思ってます！！
1601519723732434945 2022-12-10 18:10:31 +0800 <MikeOfTheSouthW> @ret_ward @TiceRichard Oh Bob 🤦‍♂️ you’ve completely embarrassed yourself here.   You do know people can see the words you type? You do realise this?  Maybe Twitter isn’t for you. 🙏
1601519721857576960 2022-12-10 18:10:31 +0800 <eZxtoY72YAoNLj3> @r_r1197 まさに今、グラウジーズ対宇都宮の試合が行われており、Twitterにもあふれています。グラは土曜日が夕方、日曜がお昼からの二連戦なので、富山到着から試合までの時間に観光、試合後はホテル駅周辺で試合談議、駅周辺の美味しいもの、日曜試合後終電まで駅周辺で楽しまれている模様
1601519721693974529 2022-12-10 18:10:31 +0800 <danialamino2> @juanjo_nzz @Licogod @migueltc98 Iba a poner un twit argumentando por qué odiar a Francia pero 

<ipython-input-3-21e8b758b635>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["traction"]= df["nlikes"] * 2 + df["nreplies"] * 3 + df["nretweets"] * 4


,tweet,nlikes,nretweets,nreplies,traction
23,"Attention! This boy here, his Facebook and Twi...",0,0,1,3
0,@TomFitton @Twitter Lol. No.,0,0,0,0
145,Elton John announces he's leaving Twitter over...,0,0,0,0
160,My Twitter Interaction Circle Generate yours ...,0,0,0,0
158,My Twitter Family: Parents: @F1onaBlackk @part...,0,0,0,0
...,...,...,...,...,...
105,On IG and Twitter they upload the trailer and ...,0,0,0,0
107,"7. Twitter can and will - both under Dorsey, A...",0,0,0,0
108,"I mean, we all know it’s because the clique mo...",0,0,0,0
111,@Crimson_Mayhem_ Twitter when people are just ...,0,0,0,0


In [98]:
sus = dedupe(df)

In [20]:
from cleantext import clean

sorted_df["tweet"] = sorted_df["tweet"].apply(clean,lower=True,no_line_breaks=True,no_urls=True,no_emails=True)


In [21]:
import re
def rmMention(passed):
    clean = re.sub(r'@\w+', '',passed)
    return clean

sorted_df["tweet"] = sorted_df["tweet"].apply(rmMention)
sorted_df

,tweet,nlikes,nretweets,nreplies,traction
23,"attention! this boy here, his facebook and twi...",0,0,1,3
0,lol. no.,0,0,0,0
145,elton john announces he's leaving twitter over...,0,0,0,0
160,my twitter interaction circle generate yours a...,0,0,0,0
158,my twitter family: parents: spouse: children: ...,0,0,0,0
...,...,...,...,...,...
105,on ig and twitter they upload the trailer and ...,0,0,0,0
107,"7. twitter can and will - both under dorsey, a...",0,0,0,0
108,"i mean, we all know it's because the clique mo...",0,0,0,0
111,twitter when people are just happy be like,0,0,0,0


In [17]:
test = "This is a test message of @mentions and seeing if it will @remove."
show = rmMention(test)
print(show)

This is a test message of  and seeing if it will .


In [6]:
clean_df.head(4)

tweet        23 attention! this boy here, his facebook and ...
nlikes       23 0 0 0 145 0 160 0 158 0 .. 105 0 107 0 108 ...
nretweets    23 0 0 0 145 0 160 0 158 0 .. 105 0 107 0 108 ...
nreplies     23 1 0 0 145 0 160 0 158 0 .. 105 0 107 0 108 ...
dtype: object

In [154]:
def s_analysis(tweet):
    analysed = analysis(tweet)
    return analysed[0]['label'], analysed[0]['score']

In [158]:
sorted_df["sentiment"], sorted_df["score"] = zip(*sorted_df["tweet"].apply(s_analysis))
sorted_df

,tweet,nlikes,nretweets,nreplies,traction,sentiment,score
1064,Causa da morte: a #CCXP22 gritando “YOU ARE BR...,2492,243,27,6037,positive,0.935704
1429,#KeanuReeves has taken the stage at #CCXP22 fo...,513,68,11,1331,neutral,0.855919
957,Why People love this guy ❤️ #KeanuReeves #Joh...,492,59,11,1253,positive,0.959775
1380,NEW: Keanu Reeves is ready to do damage in thi...,436,88,7,1245,neutral,0.618263
1523,They’re giving out this new #JohnWick4 poster ...,468,63,7,1209,neutral,0.556143
...,...,...,...,...,...,...,...
732,"In a world full of Joy Behar's, I'm going to c...",0,0,0,0,positive,0.560643
730,MLK A PORRA DO KEANU REEVES,0,0,0,0,neutral,0.847812
715,@elonmusk Elon bites the holy Apple. Apparen...,0,0,0,0,neutral,0.662710
713,@KidNamedCy @Reddit wholesome keanu reeves bab...,0,0,0,0,neutral,0.447716


In [159]:
score_df = sorted_df[sorted_df['score'] > 0.5]
score_df['sentiment'].value_counts()

positive    345
neutral     308
negative     84
Name: sentiment, dtype: int64